### Part 1

In [1]:
from pathlib import Path
import itertools
from collections import Counter, namedtuple, defaultdict
from pprint import pprint
import re
from datetime import datetime
import numpy as np
import functools
from copy import copy, deepcopy
from enum import Enum
from IPython.display import clear_output

In [2]:
data = ['x=495, y=2..7',
'y=7, x=495..501',
'x=501, y=3..7',
'x=498, y=2..4',
'x=506, y=1..2',
'x=498, y=10..13',
'x=504, y=10..13',
'y=13, x=498..504']

In [3]:
data = Path('input.txt').read_text().splitlines()

In [4]:
data2 = [sorted(l.split(', ')) for l in data]

In [5]:
MAX_X = max([max(map(int, [n[-1] for n in re.findall('x=(\d+\.\.)?(\d+)',d)])) for d in data])

In [6]:
MIN_X = min([min(map(int, [n[-1] for n in re.findall('x=(\d+\.\.)?(\d+)',d)])) for d in data])

In [7]:
MIN_Y = min([min(map(int, [n[-1] for n in re.findall('y=(\d+\.\.)?(\d+)',d)])) for d in data])

In [8]:
MAX_Y = max([max(map(int, [n[-1] for n in re.findall('y=(\d+\.\.)?(\d+)',d)])) for d in data])

In [9]:
split_data = [[list(map(int, re.findall('\d+',d))) for d in row] for row in data2]

In [10]:
symbols = {0:'.', 1:'#',2:'~',3:'+',4:'|'}

In [ ]:
#0 nothing
#1 clay
#2 water
#3 source
#4 = |
water = np.zeros((MAX_Y+1, MAX_X+1 - MIN_X))
overflow_sources = [[(0, 500-MIN_X)]]
src_index =  0

for info in split_data:
    x, *x1 = list(map(int, info[0]))
    y, *y1 = list(map(int, info[1]))
    if x1:
        water[y,x-MIN_X:x1[0]-MIN_X+1] = 1
    if y1:
        water[y:y1[0]+1,x-MIN_X] = 1
        
water[0][500-MIN_X] = 3

#0 overflowing
#1 filling
mode = 0

while True:
    clear_output()
    active_sources = copy(overflow_sources[-1])
    if not active_sources:
        overflow_sources.pop()
        active_sources = copy(overflow_sources[-1])
    num_to_complete = len(active_sources)
    num_completed = 0
    print(overflow_sources)
    print(active_sources)
    new_active_sources
    for source in active_sources:
        if water[source[0], source[1]] == 2: # ~
            #remove this source, it's underwater, go back a source
            overflow_sources[-1].remove(source)
        
        
        done = False
        #print(source)
        #calc hit spot
        stream = water[source[0]+1:, source[1]]
        first_y_stop = np.where(np.logical_and(stream>0, stream<4))[0]
        if len(first_y_stop) > 0:
            first_y_stop = first_y_stop[0]+ source[0]
        else:
            first_y_stop = water.shape[-1] + 1
            done = True
        #print(stream)
        #print(first_y_stop)

        #fill in 
        water[(source[0] + 1):(first_y_stop+1),source[1]] = 4
        if done:
            num_completed += 1
            done = False
            continue
        #print(water[(overflow_source[0] + 1):(first_y_stop+1),overflow_source[1]])

        #calculate spaces to left/right
        l_fill_arr = water[first_y_stop,:source[1]][::-1]
        
        num_l = np.where(l_fill_arr == 1)[0]
        if len(num_l) > 0:
            num_l = num_l[0]
        else:
            num_l = len(l_fill_arr)
        
        #print(num_l)

        r_fill_arr = water[first_y_stop,source[1]+1:]
        #print(r_fill_arr)
        r_available_fills = np.argmax(r_fill_arr == 1)
        num_r = np.where(r_fill_arr == 1)[0]
        if len(num_r) > 0:
            num_r = num_r[0]
        else:
            num_r = len(r_fill_arr)
        #print(num_r)

        #check for missing bottoms:
        supports = water[first_y_stop+1, (source[1]-num_l):(source[1]+num_r+1)]
        supported = np.logical_or(supports == 1, supports == 2)
        if all(supported):
            # fill in with water
            water[first_y_stop,(source[1]-num_l):(source[1]+num_r+1)] = 2
        else:
            l_capped = r_capped = False
            #print(supported)
            #print(supported[-num_r:-1])
            #print(supported[:(num_l+1)])
            first_unsupported_r = np.where(supported[-num_r:-1] == False)[0]
            if len(first_unsupported_r) > 0:
                first_unsupported_r = first_unsupported_r[0]
            else:
                first_unsupported_r = num_r-1
                r_capped = True
            #print(first_unsupported_r)
            
            first_unsupported_l = np.where(supported[:(num_l+1)][::-1] == False)[0]
            if len(first_unsupported_l) > 0:
                first_unsupported_l = first_unsupported_l[0]
            else:
                first_unsupported_l = num_l
                l_capped = True
            #print(first_unsupported_l)
            
            new_active_sources = []
                
            if not r_capped:
                new_active_sources.append((first_y_stop, source[1] + first_unsupported_r + 1))

            if not l_capped:
                new_active_sources.append((first_y_stop, source[1] - first_unsupported_l))
                
            overflow_sources.append(new_active_sources)

            water[first_y_stop,(source[1]-first_unsupported_l):(source[1]+first_unsupported_r+2)] = 4
        for row in water[max(source[0]-10,0):min(first_y_stop+5, MAX_Y),:]:
            print(''.join(symbols[t] for t in row))
        break_early = input()
        if break_early:
            break
    #print(water)
    #print(overflow_sources)
    #print()
    if num_to_complete == num_completed:
        break
    


[[(0, 242)], [(9, 240)], [(30, 230)], [(50, 228)], [(68, 230)], [(90, 228)], [(108, 239), (108, 217)], [(143, 225)]]
[(143, 225)]
..............................................................#...........#.......................................................................#.....................#......#............#.............#........#.....|.......#~~~~~~~~~~~~~~~~~~~~#..................................#.......#......................................................................................
..............................................................#...........#.......................................................................#.....................#......#............#.............#........#.....|.......#~~~~~~~~~~~~~~~~~~~~#..................................#.......#......................................................................................
..............................................................#...........#.........................................